In [35]:
# DEPENDENCIES

import pysam
import pandas as pd
import HTSeq
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
# from tqdm.notebook import tqdm, trange
import pstats
from tqdm import tqdm as tqdm
import cProfile
import plotly.io as pio
pio.renderers.default = 'notebook_connected'
import numpy as np
# from gtfparse import read_gtf


# GLOBAL VARS

FC30_DMGOTH_MAX_AS_PRIMARY_ONLY_BAMFILE ="/data2/eric/TE_LR_RNAseq/data/dmgoth101_genome_alignments/bam/FC30.against_dmgoth.filtered_max_AS.primary_only.bam"
FC29_DMGOTH_MAX_AS_PRIMARY_ONLY_BAMFILE ="/data2/eric/TE_LR_RNAseq/data/dmgoth101_genome_alignments/bam/FC29.against_dmgoth.filtered_max_AS.primary_only.bam"
GENE_ANNOTATIONS_FILE = "/data2/eric/TE_LR_RNAseq/data/dmgoth101_genome_alignments/annotations/Dm_Goth_10-1.dmel6.23LiftOff.sorted.gff"

# TE_CLASSIFICATION_FILE= "/data2/eric/TE_LR_RNAseq/TE_hierarchy.tsv"
TE_CLASSIFICATION_FILE= "/data2/eric/TE_LR_RNAseq/data/dmgoth101_genome_alignments/annotations/TE_classification.from_RM_output.csv"
# TE_ANNOTATIONS_FILE = "/data2/eric/TE_LR_RNAseq/data/dmgoth101_genome_alignments/annotations/Dm_Goth_10-1_insertions_vsTEdb.gtf"
RM_TE_ANNOTATION_FILE = "/data2/eric/TE_LR_RNAseq/data/dmgoth101_genome_alignments/annotations/dmgoth101.onecode.v3.gtf"
RM_TE_ANNOTATION_FILE_V2 = "/data2/eric/TE_LR_RNAseq/data/dmgoth101_genome_alignments/annotations/dmgoth101.onecode.v2.gtf"

# INSERTION_TABLE = "/data2/eric/TE_LR_RNAseq/data/dmgoth101_genome_alignments/insertion_table.tsv"

# GLOBAL PARAMETERS

MIN_SUB_COVERAGE = 0.1 # Threshold used to filter the features (gene or TE) mapped by a read = minimal subject coverage (nb of aligned bases / total nb of feature's bases)

In [36]:
class Gene:
	def __init__(self, chrom, start, end, gene_id):
		self.chrom = chrom
		self.start = start
		self.end = end
		self.gene_id = gene_id
		self.exons = set()
		self.mrna = set()

	def __len__(self):
		return self.end - self.start

	def __repr__(self):
		return self.gene_id


def parse_gene_annotation_line(line):
    sline = line.strip().split("\t")
    chrom = sline[0]
    start = int(sline[3])
    end = int(sline[4])
    gene_id = sline[-1].split(";")[0].split('"')[1]
    return Gene(chrom, start, end, gene_id)


def is_overlapped(start1, end1, start2, end2):
    return end1 >= start2 and end2 >= start1


def get_gene_dict(gene_annotation):
	gene_dict = {}
	with open(gene_annotation, 'r') as gene_annot:
		for line in gene_annot:
			sline = line.strip().split('\t')
			gene_id = sline[-1].split(";")[0].split('"')[1]
			if sline[2] == "gene":
				chrom = sline[0]
				new_gene = parse_gene_annotation_line(line)
				if chrom in gene_dict:
					gene_dict[chrom][new_gene.gene_id] = new_gene
				else:
					gene_dict[chrom] = {new_gene.gene_id: new_gene}
			elif sline[2] == "exon":
				gene_dict[chrom][gene_id].exons.add(
					(int(sline[3]), int(sline[4])))
			elif sline[2] == "mRNA":
				gene_dict[chrom][gene_id].mrna.add(
					(int(sline[3]), int(sline[4])))
	return gene_dict


def read_TE_classification_file(TE_classification_file):
	"""From a tsv table with 3 columns : Family, Class, Superfamily,
	return a pandas dataframe.

	Args:
		TE_classification_file (str): path to the TE classification table (tsv format)
	"""
	with open(TE_classification_file, 'r') as input:
		input.readline()
		families = list()
		superfamilies = list()
		subclasses = list()
		for line in input:
			if len(line.split("\t")) == 3:
				line = line.strip()
				family, subclass, superfamily = line.split("\t")
				families.append(family)
				superfamilies.append(superfamily)
				subclasses.append(subclass)
	TE_classification_df = pd.DataFrame(list(zip(subclasses, superfamilies, families)), columns=[
	                                    'Subclass', 'Superfamily', 'Family'])
	return TE_classification_df


class TE_feature:
	def __init__(self, chrom, start, end, gene_id, insertion_id):
		self.chrom = chrom
		self.start = start
		self.end = end
		self.insertion_id = insertion_id
		self.count = 0
		# self.family = gene_id_to_family_name(gene_id)
		self.counted_reads = set()

	def __len__(self):
		return self.end - self.start

	def __repr__(self):
		return self.insertion_id

	def is_valid(self, bam_chromosomes):
		return len(self) > 150 and self.chrom in bam_chromosomes


def build_TE(line):
    sline = line.strip().split("\t")
    chrom = sline[0]
    start = int(sline[3])
    end = int(sline[4])
    gene_id = sline[-1].split(";")[0].split('"')[1]
    insertion_id = sline[-1].strip().split('transcript_id "')[-1][:-2]

    return TE_feature(chrom, start, end, gene_id, insertion_id)


def filter_relevant_TE_feature(bamfile, TE_annotation_file, min_TE_size=150):
	"""Generate list of TE objects that will next be counted.
	TE are filtered : we discard those which are on chromosome absent from the bamfile
	and those with length (in number of base) below a certain threshold.

	Args:
		bamfile (str): path to the alignment file
		TE_annotation_file (str): path the TE annotation file (gtf format)
		min_TE_size(int): minimal number of base for a TE to be considered as valid. Default = 150
	"""
	# Enumerating chromosomes present in the bamfile
	bam_chromosomes = pysam.AlignmentFile(bamfile).references
	# Then iterating through TE_annotation_file, creating and checking TE objects
	valid_TE_list = list()
	with open(TE_annotation_file, "r") as TE_annot:
		for line in TE_annot:
			new_TE = build_TE(line)
			if new_TE.is_valid(bam_chromosomes):
				valid_TE_list.append(new_TE)
	return valid_TE_list


def regroup_TE_by_chrom(TE_feature_list):
	"""Return a dict of key:chrom and values:list of TE

	Args:
		TE_feature_list (list): flat list of TE_feature
	"""
	TE_dict = dict()
	for insertion in TE_feature_list:
		if insertion.chrom not in TE_dict:
			TE_dict[insertion.chrom] = [insertion]
		else:
			TE_dict[insertion.chrom].append(insertion)
	return TE_dict


def get_all_reads_covering_TE(TE_by_chrom, bamfile):
	"""Get a list of all reads that cover a TE.

	Args:
		TE_by_chrom (dict): dict of TE_feature ordered by chromosome
		bamfile (str): path to the alignment file
	"""
	reads_mapped_on_TE = set()
	with pysam.AlignmentFile(bamfile) as bam:
		for chrom, TE_list in tqdm(TE_by_chrom.items(), desc="Chromosome", position=0, leave=True):
			for TE in tqdm(TE_list, desc=chrom, position=0, leave=True):
				# TODO : WARNING: NEXT LINE MIGHT BREAK EVERYTHING
				reads_fetched = list(bam.fetch( contig=TE.chrom, start=TE.start, stop=TE.end) )
				overlapped_reads = [read for read in reads_fetched if read.get_overlap(TE.start, TE.end)]
				reads_mapped_on_TE.update(overlapped_reads)
				# reads_mapped_on_TE.update(list(bam.fetch(contig=TE.chrom, start=TE.start, stop=TE.end)))
	return reads_mapped_on_TE


# TODO don't forget to fix comments when done
def get_overlapped_genes(read, gene_dict):
	"""Takes a read and a dictionnary of dictionnaries (chrom -> gene_id -> [start, end])
	 and returns a list of all overlapped genes.

	Args:
		read (pysam.AlignedSegment): the read we want to map
		gene_dict (dict): a dict of dict (gene start/end by gene_id by chrom)

	Returns:
		overlapped_genes_dict (dict) : dict of all genes covered by the read with their start and end position as values

	"""
	overlapped_genes = list()
	chrom = read.reference_name
	gene_list = gene_dict[chrom].values()
	overlapped_genes = [gene for gene in gene_list if is_overlapped(
	    read.reference_start, read.reference_end, gene.start, gene.end)]
	return overlapped_genes

	# overlapped_genes = list()
	# chrom = read.reference_name
	# overlapped_genes = gene_df[(gene_df.feature == "gene") & (gene_df.seqname == chrom) & (read.reference_end >= gene_df.start) & (gene_df.end >= read.reference_start)]
	# genes_positions = [(start, end) for start, end in zip(list(overlapped_genes.start), list(overlapped_genes.end))]
	# overlapped_genes_dict = dict(zip(list(overlapped_genes.gene_id), genes_positions))
	# return overlapped_genes_dict


def get_overlapped_TE(read, TE_by_chrom):
	"""Get all TE feature that are covered by a read

	Args:
		read (pysam.AlignedSegment): the read we want to map
		TE_by_chrom (dict): dict of TE_feature with key:chrom, value:list of TE
	"""
	list_of_overlapped_TE = list()
	chrom = read.reference_name
	for insertion in TE_by_chrom[chrom]:
		if read.get_overlap(insertion.start, insertion.end):  # if at least 1 base aligned in TE
		# if read.reference_end >= insertion.start and insertion.end >= read.reference_start :
			list_of_overlapped_TE.append(insertion)
	return list_of_overlapped_TE


def get_subject_coverage(alignment, insertion):
	insertion_length = insertion.end - insertion.start
	overlap_start = max(alignment.reference_start, insertion.start)
	overlap_end = min(alignment.reference_end, insertion.end)
	overlap_length = overlap_end - overlap_start
	insertion_length = insertion.end - insertion.start
	if insertion_length != 0:
		subject_coverage = overlap_length / insertion_length
	else:
		subject_coverage = 0
	return subject_coverage

def get_nb_of_non_overlapping_bases(x_start, x_end, y_start, y_end):
	start_overlap = abs(x_start - y_start)
	end_overlap = abs(x_end - y_end)
	return start_overlap + end_overlap


def choose_feature_with_less_non_overlapping_bases(read, list_of_TE, list_of_gene):
	# Return TE if TE is the best feature, else 0
	min_nob = 999999999999
	optimal_feature = None
	for TE in list_of_TE:
		nb_of_nob = get_nb_of_non_overlapping_bases(TE.start, TE.end, read.reference_start, read.reference_end)
		if nb_of_nob < min_nob:
			min_nob = nb_of_nob
			optimal_feature = TE
	for gene in list_of_gene:
		for mrna_start, mrna_end in gene.mrna:
			nb_of_nob = get_nb_of_non_overlapping_bases(mrna_start, mrna_end, read.reference_start, read.reference_end)
			if nb_of_nob < min_nob:
				return 0
		nb_of_nob = get_nb_of_non_overlapping_bases(gene.start, gene.end, read.reference_start, read.reference_end)
		if nb_of_nob < min_nob:
			return 0
	return optimal_feature


def get_gene_exonic_positions(gene_id, gene_df):
	"""Used to get exonic positions of a gene

	Args:
		gene_id (str): the id of the gene we want to look at
		gene_df (pandas.DataFrame): a df describing the gene annotation (gtf)

	Returns:
		List of Tuples: List of tuples with start and end positions of each exons of the gene
	"""
	exons=gene_df[(gene_df.gene_id == gene_id) and (gene_df.feature == "exon")]
	exonic_positions=[(start, end)
	                   for start, end in zip(list(exons.start), list(exons.end))]
	return exonic_positions

def check_if_exonic_alignment(read, gene):  # TODO : clean comments
	"""Return True if a read has at least 1 base aligned with an exon of a gene.

	Args:
		read (pysam.AlignedSegment): a pysam read
		gene (Gene):

	Returns:
		bool: True if aligned in exonic region, else false
	"""
	exonic_positions=sorted(list(gene.exons))
	for exonStart, exonEnd in exonic_positions:
		if read.reference_end >= exonStart and exonEnd >= read.reference_start:
			return True
	return False

	# gene = gene_df[gene_df.gene_id == gene_id]
	# if str(gene.seqname) != read.reference_name or read.reference_end < gene.start:
	# 	return False
	# if read.reference_end >= gene.start and gene.end >= read.reference_start :
	# 	exonic_positions = get_gene_exonic_positions(gene_id, gene_df)
	# 	for exonStart, exonEnd in exonic_positions :
	# 		if read.get_overlap(exonStart, exonEnd):
	# 			return True
	# return False

def default_filter(alignment, insertion, subcov_threshold):
    subject_coverage=get_subject_coverage(alignment, insertion)
    nb_aligned_pairs=alignment.get_overlap(insertion.start, insertion.end)
    is_ok=(subject_coverage > subcov_threshold and nb_aligned_pairs > 1)
    return is_ok


def increment_TE_count(TE_by_chrom, reads_list, gene_dict, subcov_threshold=MIN_SUB_COVERAGE):  # TODO : fix Args !
	"""Iterate through each read that covers a TE and decide if we increment (or not)
	its read counter. This decision depends on several conditions described in the filter function.
	Return an update dict of TE, with counting + several other counters
		(1) : total_nb_of_reads covering a TE
		(2) : nb_of_chimeric_reads : count the nb of chimeric read (non-linear alignment)
		(3) : the read must not be better aligned to another feature
		(4) : TODO : CO-EXPRESSED ?

	Args:
		TE_by_chrom (dict): dict of TE_feature with key:chrom, value:list of TE
		reads_list (list): list of all the reads that cover a TE locus
		gene_annotation (dict): dict of Gene_feature with key:chrom, value:list of TE
	"""
	total_nb_of_reads=len(reads_list)
	print(total_nb_of_reads)
	nb_of_chimeric_reads=0
	nb_of_reads_covering_TE_in_exon=0
	nb_of_reads_covering_TE_in_intergenic=0
	nb_of_reads_covering_TE_in_intron=0
	nb_of_reads_filtered_by_minsubcov=0
	nb_of_reads_less_nob=0
	nb_of_read_with_one_TE=0
	nb_of_read_with_several_TE=0
	nb_of_read_aligned_on_1_base = 0
	for read in tqdm(reads_list, position=0, leave=True):
		# Counting (but not excluding) chimeric reads
		nb_of_chimeric_reads += 1 if read.has_tag("SA") else 0
		# (1) Filtering TE that are at least covered by one base.
		overlapped_TE=get_overlapped_TE(read, TE_by_chrom)
		nb_of_read_aligned_on_1_base += 1 if overlapped_TE else 0
		overlapped_genes=get_overlapped_genes(read, gene_dict)
		overlapped_in_gene_exons=[
		    gene for gene in overlapped_genes if check_if_exonic_alignment(read, gene)]
		if overlapped_genes:
			if overlapped_in_gene_exons:
				nb_of_reads_covering_TE_in_exon += 1
			else:
				nb_of_reads_covering_TE_in_intron += 1
		else:
			nb_of_reads_covering_TE_in_intergenic += 1
		if len(overlapped_TE) == 1 and len(overlapped_genes) == 0:
			nb_of_read_with_one_TE += 1
		if len(overlapped_TE) > 1 and len(overlapped_genes) == 0:
			nb_of_read_with_several_TE += 1
		# (2) filtering TE that are not covered by at least 10% of their length and at least 1 aligned base
		overlapped_TE=[TE for TE in overlapped_TE if default_filter(
		    read, TE, subcov_threshold)]
		# overlapped_TE = [TE for TE in overlapped_TE if get_subject_coverage(read, TE) >= subcov_threshold]
		nb_of_reads_filtered_by_minsubcov += 1 if overlapped_TE else 0

		# (3) filtering feature with the less non-overlapping bases
		chosen_feature=choose_feature_with_less_non_overlapping_bases(read, overlapped_TE, overlapped_in_gene_exons)
		if isinstance(chosen_feature, TE_feature):
			chosen_feature.count += 1
			nb_of_reads_less_nob += 1
			chosen_feature.counted_reads.add(read)
	counters=[total_nb_of_reads, nb_of_chimeric_reads, nb_of_reads_covering_TE_in_intergenic,
	nb_of_reads_covering_TE_in_intron, nb_of_reads_covering_TE_in_exon, nb_of_reads_filtered_by_minsubcov,
	nb_of_reads_less_nob, nb_of_read_with_one_TE, nb_of_read_with_several_TE, nb_of_read_aligned_on_1_base]
	return counters

def generate_counting(bamfile, TE_annotation_file, gene_annotation_file, TE_classification_file):
	"""Measure the expression of each TE by number of mapped reads.
	Return a dict with key = insertion_name and value = a list of mapped read

	Args:
		bamfile (str): path to the alignment file
		TE_annotation_file (str): path to the TE annotation file in gtf format
		gene_annotation_file (str): path to the gene annotation file in gtf format
		TE_classification_file (str): path to a tsv table with TE classification (family superfamily, subclass)
	"""
	pass

	# TE_classification_df = read_TE_classification_file(TE_classification_file)
	# gene_annotation = read_gtf(gene_annotation_file)
	# filtered_TE_features = filter_relevant_TE_feature(bamfile, TE_annotation_file)
	# TE_by_chrom = regroup_TE_by_chrom(filtered_TE_features)
	# # Get the list of read that covers all the insertion
	# reads_covering_TE = get_all_reads_covering_TE(TE_by_chrom, bamfile)
	# # iterating through each read and decide weither or not to increment a TE count
	# counters_results = increment_TE_count(TE_by_chrom, reads_covering_TE, gene_annotation)
	# return [TE_by_chrom, counters_results]

def get_counting_faster(bamfile, TE_annotation_file, gene_annotation_file):  # TODO : COMMENT
	print("Filtering valid TE...")
	gene_dict=get_gene_dict(gene_annotation_file)
	filtered_TE_features=filter_relevant_TE_feature(bamfile, TE_annotation_file)
	TE_by_chrom=regroup_TE_by_chrom(filtered_TE_features)
	print("Recovering reads covering TE...")
	reads_covering_TE=get_all_reads_covering_TE(TE_by_chrom, bamfile)
	print("Done")
	print("Incrementing TE counts...")
	counters_results=increment_TE_count(
	    TE_by_chrom, reads_covering_TE,  gene_dict)
	print("Done")
	return [TE_by_chrom, counters_results]

def from_raw_counting_to_df(TE_dict, TE_classification_file):
	TE_classification_df=pd.read_csv(TE_classification_file, sep="\t", names=[
	                                 "Subclass", "Superfamily", "Family", "Insertion"])
	flat_insertion_list=[]
	for insertion_list in TE_dict.values():
		flat_insertion_list += insertion_list
	insertion_dict=dict(zip([insertion.insertion_id for insertion in flat_insertion_list], [
	                    insertion for insertion in flat_insertion_list]))
	counting_list=[insertion.count for insertion in flat_insertion_list]
	counted_read_list=[
	    insertion.counted_reads for insertion in flat_insertion_list]
	counting_df=pd.DataFrame(list(zip(insertion_dict.keys(), counting_list, counted_read_list)),
				columns=['Insertion', 'Counting', 'Reads'])
	counting_df=TE_classification_df.merge(counting_df, on="Insertion")
	add_mean_subject_coverage_to_df(counting_df, insertion_dict)
	return counting_df

def get_insertion_mean_subject_coverage(insertion_id, counted_reads, insertion_dict):

    insertion=insertion_dict[insertion_id]
    mean_subject_coverage=0
    for read in counted_reads:
        mean_subject_coverage += get_subject_coverage(read, insertion)
    if len(counted_reads) == 0:
        return 0
    else:
        return mean_subject_coverage/len(counted_reads)

def add_mean_subject_coverage_to_df(counting_df, insertion_dict):
    counting_df["mean_subcov"]=counting_df.apply(lambda x: get_insertion_mean_subject_coverage(
        x['Insertion'], x['Reads'], insertion_dict), axis=1)
    return counting_df


In [64]:
def generate_TE_df(bamfile, TE_gtf, gene_gtf, TE_classification_file):
	# with cProfile.Profile() as pr:
	TE_by_chrom, counters_results = get_counting_faster(bamfile, TE_gtf, gene_gtf)
	print(counters_results)
	big_TE_df = from_raw_counting_to_df(TE_by_chrom, TE_classification_file)
	return [big_TE_df, counters_results]

# FC29_TE_df, FC29_counters = generate_TE_df(FC29_DMGOTH_MAX_AS_PRIMARY_ONLY_BAMFILE, RM_TE_ANNOTATION_FILE, GENE_ANNOTATIONS_FILE, TE_CLASSIFICATION_FILE)
# FC30_TE_df, FC30_counters = generate_TE_df(FC30_DMGOTH_MAX_AS_PRIMARY_ONLY_BAMFILE, RM_TE_ANNOTATION_FILE, GENE_ANNOTATIONS_FILE, TE_CLASSIFICATION_FILE)

FC29_TE_df, FC29_counters = generate_TE_df(FC29_DMGOTH_MAX_AS_PRIMARY_ONLY_BAMFILE, RM_TE_ANNOTATION_FILE_V2, GENE_ANNOTATIONS_FILE, TE_CLASSIFICATION_FILE)
FC30_TE_df, FC30_counters = generate_TE_df(FC30_DMGOTH_MAX_AS_PRIMARY_ONLY_BAMFILE, RM_TE_ANNOTATION_FILE_V2, GENE_ANNOTATIONS_FILE, TE_CLASSIFICATION_FILE)

		
		# stats = pstats.Stats(pr)
		# stats.sort_stats(pstats.SortKey.TIME)
		# stats.dump_stats(filename="profiling_test.prof")



## counters_results : we got for FC30 with annot.v3 : [7332, 2085, 705, 382, 6245, 5946, 1338, 534, 171]
## and for FC29 : [22232, 5854, 6075, 2139, 14018, 17106, 8809, 3997, 2078]

Filtering valid TE...
Recovering reads covering TE...


Chromosome: 100%|██████████| 6/6 [00:40<00:00,  6.76s/it]


Done
Incrementing TE counts...
7239


100%|██████████| 7239/7239 [03:45<00:00, 32.11it/s]


Done
[7239, 2054, 705, 326, 6208, 5852, 1322, 535, 170, 7239]


In [65]:
def save_counting_df(counting_df, csv_file):
    counting_df.to_csv(csv_file, sep = '\t', index=False)

# save_counting_df(FC29_counting_df, "FC29_counting_df.v2.tsv")
# save_counting_df(FC30_counting_df, "FC30_counting_df.v2.tsv")

### SAVING COUNTING_DF WITH CHIMERIC READS EXCLUDED
# save_counting_df(FC29_counting_df, "FC29_counting_df.v2.chimeric_reads_excluded.tsv")
# save_counting_df(FC30_counting_df, "FC30_counting_df.v2.chimeric_reads_excluded.tsv")

### SAVING COUNTING_DF co_expressed
save_counting_df(FC29_TE_df, "/data2/eric/TE_LR_RNAseq/countings/FC29_counting_df.v2.tsv")
save_counting_df(FC30_TE_df, "/data2/eric/TE_LR_RNAseq/countings/FC30_counting_df.v2.tsv")

In [42]:
### with old annotations

def create_TE_hierarchy(te_hierarchy_file):
	family_list = []
	superfamily_list = []
	subclass_list = []
	with open(te_hierarchy_file,'r') as input:
		input.readline()
		for line in input:
			if len(line.split()) == 3:
				line = line.strip()
				family, superfamily, subclass = line.split("\t")
				family_list.append(family)
				superfamily_list.append(superfamily)
				subclass_list.append(subclass)
	hierarchy_df = pd.DataFrame(list(zip(subclass_list, superfamily_list, family_list)), columns =['Subclass', 'Superfamily', 'Family'])
	return hierarchy_df	
    
def gene_id_to_family_name(gene_id):
    family_name = gene_id
    suffix_list = ["_I", "-I_DM", "_I_DM", "_LTR", "-LTR_DM", "_LTR_DM"]
    for suffix in suffix_list:
        if gene_id[-len(suffix):] == suffix:
            family_name = gene_id[:-len(suffix)]
            break
    return family_name

class TE_feature:
    def __init__(self, chrom, start, end, gene_id, insertion_id):
        self.chrom = chrom
        self.start = start
        self.end = end
        # self.gene_id = gene_id
        self.insertion_id = insertion_id
        self.count = 0
        self.family = gene_id_to_family_name(gene_id)
        self.counted_reads = {}
    
    def __len__(self):
        return self.end - self.start
    
    def __repr__(self):
        return self.insertion_id
    
    def is_valid_feature(self):
        return len(self) > 150
    
def parse_TE_annotation_line(line):
    sline = line.strip().split("\t")
    chrom = sline[0]
    start = int(sline[3])
    end = int(sline[4])
    gene_id = sline[-1].split(";")[0].split('"')[1]
    insertion_id = sline[-1].strip().split('transcript_id "')[-1][:-2]

    return TE_feature(chrom, start, end, gene_id, insertion_id)

def parse_TE_annotation_file(annotations):
    ### Read an annotation file and return a dict of filled TE feature per chromosome
    dict_of_valid_TE = {}
    with open(annotations, 'r') as annot:
        for line in annot :
            new_feature = parse_TE_annotation_line(line)
            if new_feature.is_valid_feature and new_feature.chrom in VALID_CHROM_LIST:
                if new_feature.chrom not in dict_of_valid_TE:
                    dict_of_valid_TE[new_feature.chrom] = [new_feature]
                else :
                    dict_of_valid_TE[new_feature.chrom].append(new_feature)
    for chrom in dict_of_valid_TE :
        dict_of_valid_TE[chrom].sort(key=lambda x: x.start, reverse=False)
    return dict_of_valid_TE

def get_reads_mapped_on_TE_feature(alignment_file, TE_annotation):
    ### return a set of all reads that map a TE feature
    reads_mapped_on_TE = []
    dict_of_valid_TE = parse_TE_annotation_file(TE_annotation)
    with pysam.AlignmentFile(alignment_file, 'rb') as bam:
        for chrom, insertion_list in tqdm(dict_of_valid_TE.items(), desc="Chromosome", position=0, leave=True):
            for insertion in tqdm(insertion_list, desc =chrom, position=0, leave=True):
                try :
                    reads_mapped_on_TE += list(bam.fetch(contig=insertion.chrom, start=insertion.start, stop=insertion.end))
                except ValueError:
                    # print("{} chromosome not found in bamfile ! Skipping...".format(insertion.chrom))
                    pass
    return set(reads_mapped_on_TE)

In [6]:
class Gene:
    def __init__(self, chrom, start, end, gene_id):
        self.chrom = chrom
        self.start = start
        self.end = end
        self.gene_id = gene_id
    
    def __len__(self):
        return self.end - self.start
    
    def __repr__(self):
        return self.gene_id

def parse_gene_annotation_line(line):
    sline = line.strip().split("\t")
    chrom = sline[0]
    start = int(sline[3])
    end = int(sline[4])
    gene_id = sline[-1].split(";")[0].split('"')[1]
    return Gene(chrom, start, end, gene_id)


def is_overlapped(start1, end1, start2, end2):
    return end1 >= start2 and end2 >= start1

def get_overlapped_TE(read, dict_of_valid_TE):
    list_of_overlapped_TE =[]
    chrom = read.reference_name
    for insertion in dict_of_valid_TE[chrom]:
        if read.reference_end < insertion.start:
            continue
        if read.reference_end >= insertion.start and insertion.end >= read.reference_start :
            list_of_overlapped_TE.append(insertion)
    return list_of_overlapped_TE

def get_dict_of_overlapped_gene(dict_of_valid_TE, gene_annotation):
    dict_of_overlapped_gene = {}
    with open(gene_annotation, 'r') as gene_annot:
        for line in gene_annot:
            sline = line.strip().split('\t')
            if sline[2] == "gene" :
                chrom = sline[0]
                gene_start = int(sline[3])
                gene_stop = int(sline[4])
                if chrom in dict_of_valid_TE:
                    for insertion in dict_of_valid_TE[chrom]:
                        if is_overlapped(insertion.start, insertion.end, gene_start, gene_stop):
                            new_gene = parse_gene_annotation_line(line)
                            if chrom in dict_of_overlapped_gene:
                                dict_of_overlapped_gene[chrom].append(new_gene)
                            else:
                                dict_of_overlapped_gene[chrom] = [new_gene]
    return dict_of_overlapped_gene   
    
def get_overlapped_gene(read, dict_of_overlapped_gene): # TODO : is it taking chrom into account ??
    list_of_overlapped_gene =[]
    for gene in dict_of_overlapped_gene[read.reference_name]:
        if read.reference_end < gene.start:
            continue
        if is_overlapped(read.reference_start, read.reference_end, gene.start, gene.end):
            list_of_overlapped_gene.append(gene)
    return list_of_overlapped_gene

def choose_optimal_feature(read, list_of_overlapped_feature):
    min_dist = 999999999999
    optimal_feature = None
    for overlapped_feature in list_of_overlapped_feature:
        start_dist = abs(overlapped_feature.start - read.reference_start)
        end_dist = abs(overlapped_feature.end - read.reference_end)
        total_dist = start_dist + end_dist
        if total_dist < min_dist:
            min_dist = total_dist
            optimal_feature = overlapped_feature
    return optimal_feature

def get_subject_coverage(alignment, insertion):
    overlap_start = max(alignment.reference_start, insertion.start)
    overlap_end = min(alignment.reference_end, insertion.end)
    overlap_length = overlap_end - overlap_start
    insertion_length = insertion.end - insertion.start
    if insertion_length != 0 :
        subject_coverage = overlap_length / insertion_length
    else :
        subject_coverage = 0
    return subject_coverage

def default_filter(alignment, insertion):
    subject_coverage = get_subject_coverage(alignment, insertion)
    nb_aligned_pairs =  alignment.get_overlap(insertion.start, insertion.end)
    is_ok = (subject_coverage > 0.1 and nb_aligned_pairs > 1)
    return is_ok

def get_exceeding_alignment_length(alignment, insertion):
    overflow_length = 0
    if alignment.reference_start < insertion.start:
        overflow_length += insertion.start -alignment.reference_start
    if alignment.reference_end > insertion.end:
        overflow_length += alignment.reference_end - insertion.end
    return overflow_length

def is_co_expressed(alignment, list_of_overlapped_TE, list_of_overlapped_gene):
    # We need to check if the alignment get at least 10% of its aligned base on a TE and 10% on a gene
    # Subject coverage of 10% on a TE and on a gene !
    overlapping_TE = False
    overlapping_gene = False
    for TE in list_of_overlapped_TE :
        if get_subject_coverage(alignment, TE) > 0.1:
            overlapping_TE = True
            break
    for gene in list_of_overlapped_gene :
        if get_subject_coverage(alignment, gene) > 0.1:
            overlapping_gene = True
            break
    return overlapping_gene and overlapping_TE

def filter_out_co_expressed(alignment, insertion):
    exceeding_length = get_exceeding_alignment_length(alignment, insertion)
    alignment_length = alignment.reference_end - alignment.reference_start
    is_ok = (exceeding_length < (0.1 * alignment_length))
    return is_ok

def map_reads_to_TE(reads_mapped_on_TE_feature, bamfile, TE_annotations, gene_annotations):
    insertion_countings = {}
    dict_of_valid_TE = parse_TE_annotation_file(TE_annotations)
    dict_of_overlapped_gene = get_dict_of_overlapped_gene(dict_of_valid_TE, gene_annotations)
    for read in tqdm(reads_mapped_on_TE_feature, position=0, leave=True):
        ### CHIMERIC READS GOT AN "SA" FLAG : WE ARE FILTERING THEM HERE 
        if not read.has_tag("SA") :
            list_of_overlapped_gene = get_overlapped_gene(read, dict_of_overlapped_gene)
            list_of_overlapped_TE = get_overlapped_TE(read, dict_of_valid_TE)
            best_feature = choose_optimal_feature(read, list_of_overlapped_TE + list_of_overlapped_gene)
            if isinstance(best_feature, TE_feature):
                ################### FILTERS GO HERE ########################
                # if default_filter(read, best_feature) and filter_out_co_expressed(read, best_feature):
                if default_filter(read, best_feature) and filter_out_co_expressed(read, best_feature) and is_co_expressed(read, list_of_overlapped_TE, list_of_overlapped_gene):
                    best_feature.count += 1
                    best_feature.counted_reads[read.query_name] = read
    for insertion_list in dict_of_valid_TE.values():
        for insertion in insertion_list:
            if insertion.family not in insertion_countings:
                insertion_countings[insertion.family] = {insertion.insertion_id : insertion}
            else :
                insertion_countings[insertion.family][insertion.insertion_id] = insertion
    return insertion_countings

In [44]:
def get_reads_counted_on_feature(feature_id, reads_mapped_on_TE_feature, bamfile, TE_annotations, gene_annotations):
    read_list = []
    dict_of_valid_TE = parse_TE_annotation_file(TE_annotations)
    dict_of_overlapped_gene = get_dict_of_overlapped_gene(dict_of_valid_TE, gene_annotations)
    feature_chrom = None
    for chrom, feature_list in dict_of_valid_TE.items():
        if feature_id in [f.insertion_id for f in feature_list]:
            feature_chrom = chrom
    if feature_chrom == None :
        return 0
    for read in tqdm(reads_mapped_on_TE_feature, position=0, leave=True):
        if read.reference_name == feature_chrom:
            list_of_overlapped_gene = get_overlapped_gene(read, dict_of_overlapped_gene)
            list_of_overlapped_TE = get_overlapped_TE(read, dict_of_valid_TE)
            best_feature = choose_optimal_feature(read, list_of_overlapped_TE + list_of_overlapped_gene)
            if isinstance(best_feature, TE_feature):
                if best_feature.insertion_id == feature_id:
                    if default_filter(read, best_feature) and filter_out_co_expressed(read, best_feature):
                        read_list.append(read)
    return read_list

def from_counting_to_dataframe(insertion_countings):
    subclass_list = []
    superfamily_list = []
    family_list = []
    insertion_list = []
    counting_list = []
    counted_reads = []
    for family, family_insertion_dict in insertion_countings.items() :
        try:
            subclass = HIERARCHY_DF.loc[HIERARCHY_DF['Family'] == family, 'Subclass'].values[0]
            superfamily = HIERARCHY_DF.loc[HIERARCHY_DF['Family'] == family, 'Superfamily'].values[0]
        except KeyError:
            print(family + " hierarchy unknown in the consensus file : tagged as Unknown...")
            subclass = "Unknown"
            superfamily = "Unknown"
        for insertion_id, insertion in family_insertion_dict.items():
            family_list.append(family)
            insertion_list.append(insertion_id)
            counting_list.append(insertion.count)
            superfamily_list.append(superfamily)
            subclass_list.append(subclass)
            counted_reads.append(insertion.counted_reads)
    df = pd.DataFrame(list(zip(subclass_list, superfamily_list, family_list, insertion_list, counting_list, counted_reads)),
               columns =['Subclass', 'Superfamily', "Family", "Insertion", "Counting", "Reads"])
    return df

In [45]:
def get_insertion_mean_subject_coverage(insertion_id, counted_reads):
    insertion = DICT_OF_INSERTION[insertion_id]
    mean_subject_coverage = 0
    for read in counted_reads.values():
        mean_subject_coverage += get_subject_coverage(read, insertion)
    if len(counted_reads) == 0 :
        return 0
    else:
        return mean_subject_coverage/len(counted_reads)

def add_mean_subject_coverage_to_df(counting_df):
    counting_df["mean_subcov"] = counting_df.apply(lambda x: get_insertion_mean_subject_coverage(x['Insertion'], x['Reads']), axis=1)
    return counting_df

def count_TE_feature(alignment_file, TE_annotation, gene_annotation):
    reads_mapped_on_TE_feature = get_reads_mapped_on_TE_feature(alignment_file, TE_annotation)
    insertion_countings = map_reads_to_TE(reads_mapped_on_TE_feature, alignment_file, TE_annotation, gene_annotation)
    counting_df = from_counting_to_dataframe(insertion_countings)
    add_mean_subject_coverage_to_df(counting_df)
    return counting_df




# with cProfile.Profile() as pr:
#     FC30_counting_df = count_TE_feature(FC29_DMGOTH_MAX_AS_BAMFILE, REPEATMASKER_TE_ANNOTATIONS, DMGOTH_GENE_ANNOTATIONS)
# stats = pstats.Stats(pr)
# stats.sort_stats(pstats.SortKey.TIME)
# stats.dump_stats(filename="profiling_test.prof")
# pd.set_option('display.max_rows', 100)
# FC30_counting_df.sort_values(by=['Counting'], ascending=False).head(100)

In [15]:
# HIERARCHY_DF  = create_TE_hierarchy(TE_HIERARCHY_FILE)
# FC30_counting_df = count_TE_feature(FC30_DMGOTH_MAX_AS_BAMFILE, NEW_ANNOTATION_FILE, DMGOTH_GENE_ANNOTATIONS)
# FC29_counting_df = count_TE_feature(FC29_DMGOTH_MAX_AS_BAMFILE, NEW_ANNOTATION_FILE, DMGOTH_GENE_ANNOTATIONS)


In [16]:
# FC29_counting_df.sort_values(by=['Counting'], ascending=False).head(100)

In [29]:
def save_counting_df(counting_df, csv_file):
    counting_df.to_csv(csv_file, sep = '\t', index=False)

# save_counting_df(FC29_counting_df, "FC29_counting_df.v2.tsv")
# save_counting_df(FC30_counting_df, "FC30_counting_df.v2.tsv")

### SAVING COUNTING_DF WITH CHIMERIC READS EXCLUDED
# save_counting_df(FC29_counting_df, "FC29_counting_df.v2.chimeric_reads_excluded.tsv")
# save_counting_df(FC30_counting_df, "FC30_counting_df.v2.chimeric_reads_excluded.tsv")

### SAVING COUNTING_DF co_expressed
save_counting_df(FC29_TE_df, "/data2/eric/TE_LR_RNAseq/countings/FC29_counting_df.v3.tsv")
save_counting_df(FC30_TE_df, "/data2/eric/TE_LR_RNAseq/countings/FC30_counting_df.v3.tsv")

In [47]:
### Get expression ratio for each insertion (chimeric excluded)

## First, we merge male and female dataset : 

# print(FC29_no_chimere_counting_df.head())
# print(FC29_counting_df.head())

expression_df = FC29_counting_df.copy().drop(columns=["mean_subcov", "Reads"])
expression_df["Counting_no_chimere"] = FC29_no_chimere_counting_df["Counting"]

excluded_families = ["TAHRE", "TART_B1", "TART-A", "HETA"]
# excluded_families = []

expression_df = expression_df[~expression_df["Family"].isin(excluded_families)]

nb_of_reads = sum(expression_df["Counting"])
nb_of_reads_no_chimere = sum(expression_df["Counting_no_chimere"])
# print(expression_df)
def get_expression_ratio(counting):
	return counting / nb_of_reads

def get_expression_ratio_bis(counting):
	return counting / nb_of_reads

expression_df["Expression"] = expression_df["Counting"].apply(get_expression_ratio)
expression_df["Expression_no_chimere"] = expression_df["Counting_no_chimere"].apply(get_expression_ratio_bis)

# get length of each insertion
length_list = []

for i in expression_df["Insertion"].values:
	length_list.append(DICT_OF_INSERTION[i].end - DICT_OF_INSERTION[i].start + 1)
expression_df["Insertion_length"] = length_list


save_counting_df(expression_df, "FC29_expression_df.tsv")



In [44]:
import plotly.express as px
df = expression_df.copy()
df = df[df["Counting"] > 2]
df["exp_diff"] = df["Expression"] - df["Expression_no_chimere"]
fig = px.scatter(df, y="exp_diff", x="Insertion", color="Family", size="Insertion_length")
# fig.update_traces(marker_size=10)
fig.show()

In [51]:
import plotly.express as px
df = expression_df.copy()
df = df[df["Counting"] > 2]
df["count_diff"] = df["Counting"] - df["Counting_no_chimere"]
fig = px.scatter(df, y="count_diff", x="Insertion", color="Family", size="Insertion_length")
# fig.update_traces(marker_size=10)
fig.show()

### IMPORT COUNTINGS

In [67]:

# saved_FC29_counting_df = pd.read_csv("/data2/eric/TE_LR_RNAseq/countings/FC29_counting_df.v2.tsv", sep = "\t")
# filtered_FC29_counting_df = saved_FC29_counting_df[saved_FC29_counting_df['Counting'] != 0]
# co_expressed_FC29_counting_df = pd.read_csv("FC29_counting_df.v2.co_expressed.tsv", sep = "\t")

# saved_FC30_counting_df = pd.read_csv("/data2/eric/TE_LR_RNAseq/countings/FC30_counting_df.v2.tsv", sep = "\t")
# filtered_FC30_counting_v2_df = saved_FC30_counting_df[saved_FC30_counting_df['Counting'] != 0]
# # co_expressed_FC30_counting_df = pd.read_csv("FC30_counting_df.v2.co_expressed.tsv", sep = "\t")

saved_FC29_counting_df = pd.read_csv("/data2/eric/TE_LR_RNAseq/countings/FC29_counting_df.v2.tsv", sep = "\t")
filtered_FC29_counting_df = saved_FC29_counting_df[saved_FC29_counting_df['Counting'] >= 1]

saved_FC30_counting_df = pd.read_csv("/data2/eric/TE_LR_RNAseq/countings/FC30_counting_df.v2.tsv", sep = "\t")
filtered_FC30_counting_df = saved_FC30_counting_df[saved_FC30_counting_df['Counting'] >= 1]

In [32]:
filtered_FC29_counting_df.sort_values(by=['Counting'], ascending=False).head(20)

,Subclass,Superfamily,Family,Insertion,Counting,Reads,mean_subcov
7161,LTR,Gypsy,Gypsy10_I,Gypsy10_I$3R_RaGOO$761442$762629,489,{<pysam.libcalignedsegment.AlignedSegment obje...,0.447665
1860,LINE,I-Jockey,TAHRE,TAHRE$2R_RaGOO$1145909$1151824,269,{<pysam.libcalignedsegment.AlignedSegment obje...,0.171054
4388,LINE,I-Jockey,TART_B1,TART_B1$2R_RaGOO$23920095$23923160,251,{<pysam.libcalignedsegment.AlignedSegment obje...,0.629438
1458,LTR,Gypsy,NOMAD_I,NOMAD_I$2R_RaGOO$383343$386109,189,{<pysam.libcalignedsegment.AlignedSegment obje...,0.356018
4393,LINE,I-Jockey,HETA,HETA$3L_RaGOO$18403$20679,177,{<pysam.libcalignedsegment.AlignedSegment obje...,0.378471
4392,LINE,I-Jockey,HETA,HETA$3L_RaGOO$12349$18402,152,{<pysam.libcalignedsegment.AlignedSegment obje...,0.373199
4394,LINE,I-Jockey,TAHRE,TAHRE$3L_RaGOO$20705$26581,135,{<pysam.libcalignedsegment.AlignedSegment obje...,0.669291
1151,LTR,Gypsy,NOMAD_I,NOMAD_I$2L_RaGOO$22581023$22587526,132,{<pysam.libcalignedsegment.AlignedSegment obje...,0.245340
2348,LTR,Gypsy,Gypsy_I,Gypsy_I$2R_RaGOO$2048940$2052346,131,{<pysam.libcalignedsegment.AlignedSegment obje...,0.160444
10369,LINE,I-Jockey,Jockey2,Jockey2$X_RaGOO$21275337$21276483,120,{<pysam.libcalignedsegment.AlignedSegment obje...,0.736249


In [33]:
def draw_icicle(df):
    fig = px.icicle(df, path=['Subclass', 'Superfamily', 'Family', 'Insertion'], values='Counting',
                    color='mean_subcov',
                    hover_data=['Counting'],
                    color_continuous_scale='RdBu',
                    color_continuous_midpoint=np.average(df['mean_subcov'],weights=df['Counting'])
                    )
    fig.update_traces(root_color="lightgrey")
    fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
    fig.show()

def draw_sunburst(df):
    fig = px.sunburst(df, path=['Subclass', 'Superfamily', 'Family', 'Insertion'], values='Counting',
                    color='mean_subcov',
                    hover_data=['Counting'],
                    color_continuous_scale='Plasma',
                    # color_continuous_scale='RdBu',
                    # color_continuous_midpoint=np.average(df['mean_subcov'],weights=df['Counting'])
                    color_continuous_midpoint=np.average(0.5)
                    )
    fig.update_traces(root_color="lightgrey")
    fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
    fig.show()

In [34]:
draw_sunburst(filtered_FC29_counting_df)

In [68]:
draw_sunburst(filtered_FC30_counting_df)


In [104]:
draw_sunburst(filtered_FC30_counting_df)


In [55]:
pd.set_option('display.max_rows', 100)
# filtered_FC29_counting_df.sort_values(by=['Counting'], ascending=False).head(100)
# print(saved_FC29_counting_df[saved_FC29_counting_df['Insertion'] == "Gypsy_I_2R_RaGOO_2048940_2052346"])
# print(saved_FC29_counting_df[saved_FC29_counting_df['Insertion'] == "Gypsy6_LTR_2R_RaGOO_2049422_2049472"])

In [106]:
copia_df

,Subclass,Superfamily,Family,Insertion,Counting,Reads,mean_subcov
10,LTR,Copia,Copia_LTR,Copia_LTR$2L_RaGOO$12650$17604,1,{<pysam.libcalignedsegment.AlignedSegment obje...,0.944489
93,LTR,Copia,Copia_LTR,Copia_LTR$2L_RaGOO$9004503$9009641,4,{<pysam.libcalignedsegment.AlignedSegment obje...,0.742215
143,LTR,Copia,Copia_LTR,Copia_LTR$2L_RaGOO$13770083$13775227,8,{<pysam.libcalignedsegment.AlignedSegment obje...,0.804335
151,LTR,Copia,Copia_LTR,Copia_LTR$2L_RaGOO$14753817$14758950,16,{<pysam.libcalignedsegment.AlignedSegment obje...,0.760849
658,LTR,Copia,Copia_LTR,Copia_LTR$2L_RaGOO$21771835$21776956,16,{<pysam.libcalignedsegment.AlignedSegment obje...,0.894515
4341,LTR,Copia,Copia_LTR,Copia_LTR$2R_RaGOO$20482857$20488005,10,{<pysam.libcalignedsegment.AlignedSegment obje...,0.676748
4513,LTR,Copia,Copia_LTR,Copia_LTR$3L_RaGOO$10022428$10027467,33,{<pysam.libcalignedsegment.AlignedSegment obje...,0.822668
4514,LTR,Copia,Copia_LTR,Copia_LTR$3L_RaGOO$10052506$10057644,24,{<pysam.libcalignedsegment.AlignedSegment obje...,0.836756
4599,LTR,Copia,Copia_LTR,Copia_LTR$3L_RaGOO$17623155$17628299,15,{<pysam.libcalignedsegment.AlignedSegment obje...,0.863945
7498,LTR,Copia,Copia_LTR,Copia_LTR$3R_RaGOO$1372041$1377188,6,{<pysam.libcalignedsegment.AlignedSegment obje...,0.486627


In [40]:
copia_df = saved_FC30_counting_df[saved_FC30_counting_df.Family == "Copia_LTR"]
fig = px.violin(copia_df, y="Counting", box = True, points = 'all')
fig.show()

In [11]:
pd.set_option('display.max_rows', 100)
filtered_FC30_counting_df.sort_values(by=['Counting'], ascending=False).head(100)
print(saved_FC30_counting_df[saved_FC30_counting_df['Insertion'] == "Gypsy6_LTR_2R_RaGOO_2049422_2049472"])

      Subclass Superfamily  Family                            Insertion  \
12296      LTR       Gypsy  Gypsy6  Gypsy6_LTR_2R_RaGOO_2049422_2049472   

       Counting Reads  mean_subcov  
12296         0    {}          0.0  


In [88]:
def get_insertion_length(insertion_name):
	start, end = insertion_name.split('_')[-2:]
	return int(end) - int(start) + 1

def create_summary_table(counting_table):
	subclass_list = []
	superfamily_list = []
	family_list = counting_table["Family"].unique()
	min_TE_length_list = []
	max_TE_length_list = []
	counting_list = []
	nb_expressed_insertion_list = []
	nb_insertion_list = []
	most_expressed_insertion_list = []
	for family in family_list :
		family_df = counting_table[counting_table["Family"] == family]
		subclass_list.append(family_df["Subclass"].values[0])
		superfamily_list.append(family_df["Superfamily"].values[0])
		min_TE_length = 9999999
		max_TE_length = 0
		for insertion_name in family_df["Insertion"]:
			insertion_length = get_insertion_length(insertion_name)
			if insertion_length < min_TE_length :
				min_TE_length = insertion_length
			if insertion_length > max_TE_length :
				max_TE_length = insertion_length
		min_TE_length_list.append(min_TE_length)
		max_TE_length_list.append(max_TE_length)
		nb_insertion_list.append(len(family_df["Insertion"]))
		counting_list.append(family_df["Counting"].sum())
		nb_expressed_insertion_list.append(len(family_df[family_df["Counting"] > 0]))
		most_expressed_insertion_list.append(family_df[family_df["Counting"] == family_df["Counting"].max()]["Insertion"].values[0])
    
	summary_df = pd.DataFrame(list(zip(subclass_list, superfamily_list, family_list, min_TE_length_list, max_TE_length_list, counting_list, nb_insertion_list, nb_expressed_insertion_list, most_expressed_insertion_list)), columns=["Subclass", "Superfamily", "Family", "Min_TE_length","Max_TE_length", "Counts", "nb_of_insertion", "nb_of_expressed_insertion", "Most_expressed_insertion"])
	return summary_df


def merge_female_and_male_df(female_counting, male_counting):
	female_df = create_summary_table(female_counting)
	male_df = create_summary_table(male_counting)
	hierarchy_cols = female_df[["Subclass", "Superfamily", "Family", "nb_of_insertion", "Min_TE_length","Max_TE_length"]]
	hierarchy_df = hierarchy_cols.copy()
	full_summary_df = hierarchy_df.merge(female_df.drop(columns=["Subclass", "Superfamily", "Min_TE_length","Max_TE_length", "nb_of_insertion"]), on='Family').merge(male_df.drop(columns=["Subclass", "Superfamily", "Min_TE_length","Max_TE_length", "nb_of_insertion"]), on='Family')
	full_summary_df.columns = ["Subclass", "Superfamily", "Family", "nb_of_insertion", "Min_TE_length", "Max_TE_length", "Female_Counts", "Female_nb_of_expressed_insertion", "Female_most_expressed_insertion", "Male_Counts", "Male_nb_of_expressed_insertion", "Male_most_expressed_insertion"]

	return full_summary_df

merged_full_summary_df = merge_female_and_male_df(saved_FC30_counting_df, saved_FC29_counting_df)
# print(merged_full_summary_df)
save_counting_df(merged_full_summary_df, "merged_full_summary.v2.tsv")

# def create_table_from_counting(female_counting, male_counting, hierarchy):
#     male_counting = male_counting.drop(columns=["Min_TE_length", "Max_TE_length", "nb_of_insertion"])
#     TE_data_table = hierarchy.merge(female_counting, on='Family').merge(male_counting, on='Family')
#     TE_data_table.columns = ["Subclass", "Superfamily", "Family", "Min_TE_length", "Max_TE_length", "Female_Counts", "Nb_of_insertion", "Female_nb_of_expressed_insertion", "Female_most_expressed_insertion", "Male_Counts", "Male_nb_of_expressed_insertion", "Male_most_expressed_insertion"]
#     TE_data_table = TE_data_table[["Subclass", "Superfamily", "Family", "Nb_of_insertion", "Min_TE_length", "Max_TE_length", "Female_Counts", "Female_nb_of_expressed_insertion", "Female_most_expressed_insertion", "Male_Counts", "Male_nb_of_expressed_insertion", "Male_most_expressed_insertion"]]
#     return TE_data_table

In [ ]:
def from_counting_to_df(counting_dict):
    family_list = []
    min_TE_length_list = []
    max_TE_length_list = []
    counting_list = []
    nb_expressed_insertion_list = []
    nb_insertion_list = []
    most_expressed_insertion_list = []
    for family, insertion_list in counting_dict.items():
        family_list.append(family)
        nb_insertion_list.append(len(insertion_list))
        nb_expressed_insertion = 0
        min_TE_length = 9999999999
        max_TE_length = 0
        family_counts = 0
        most_expressed_insertion = None
        max_count = 0
        for insertion in insertion_list :
            insertion_length = insertion.end - insertion.start
            min_TE_length = min(min_TE_length, insertion_length)
            max_TE_length = max(max_TE_length, insertion_length)
            family_counts += insertion.counts
            max_count = max(max_count,insertion.counts)
            if insertion.counts >= max_count :
                most_expressed_insertion = insertion.id
            if insertion.counts > 0 :
                nb_expressed_insertion += 1
        nb_expressed_insertion_list.append(nb_expressed_insertion)
        min_TE_length_list.append(min_TE_length)
        max_TE_length_list.append(max_TE_length)
        counting_list.append(family_counts)
        most_expressed_insertion_list.append(most_expressed_insertion)
        
    counting_df = pd.DataFrame(list(zip(family_list, min_TE_length_list, max_TE_length_list, counting_list, nb_insertion_list, nb_expressed_insertion_list, most_expressed_insertion_list)), columns=["Family", "Min_TE_length","Max_TE_length", "Counts", "nb_of_insertion", "nb_of_expressed_insertion", "Most_expressed_insertion"])
    return counting_df

In [37]:
### Export reads mapped on the 3 expressed insertions of POGO.

#### Get ID of reads mapped on POGO

# insertion = "POGO_3L_RaGOO_9733928_9735150"
# insertion = "POGO_X_RaGOO_21863530_21864880"
# insertion = "POGO_2R_RaGOO_7201268_7202754"

# mapped_reads = saved_FC30_counting_df[saved_FC30_counting_df["Insertion"] == insertion]["Reads"]
# mapped_reads = mapped_reads.iloc[0].split("'")
# reads_IDs = []
# for i, j in enumerate(mapped_reads):
# 	if i % 2 :
# 		reads_IDs.append(j)
# test =  pysam.AlignmentFile(FC30_DMGOTH_MAX_AS_BAMFILE, "r")
# read_it = test.fetch(contig = "2R_RaGOO", start = 7201268, end = 7202754)
# with open(insertion + ".mapped_read.fasta", 'w') as output:
# 	for read in read_it:
# 		if read.query_name in reads_IDs:
# 			if read.query_sequence != None:
# 				new_line = "> " + read.query_name + "\n" + read.query_sequence + "\n"
# 				output.write(new_line)


